In [52]:
from ttide.t_tide import t_tide
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

In [5]:
#load dataset, Time(UTC),prs(m),rad(m)
df2 = pd.read_csv('data2y.csv',sep=',', header=0, index_col=0, squeeze=True, parse_dates=True)
#non-null object to float64
df2['prs(m)'] = pd.to_numeric(df2['prs(m)'], errors='coerce')
#prs(m) looks more stable
df2.drop(df2.columns[[1]], axis=1, inplace=True)

s = df2.interpolate()
s.head()

,prs(m)
Time(UTC),
2016-12-31 00:00:52,2.369
2016-12-31 00:01:53,2.360
2016-12-31 00:02:52,2.369
2016-12-31 00:03:53,2.361
2016-12-31 00:04:53,2.392


In [14]:
df2.index[0]

Timestamp('2016-12-31 00:00:52')

In [105]:
out = t_tide(s['prs(m)'], dt=0.0166667, stime=df2.index[0], lat=-33.02730833)
#out = t_tide(dfp['prs(m) predictions'], dt=0.016, lat=np.array(-33.02730833))

C:\Users\sergi\Anaconda3\lib\site-packages\ttide-0.3_exp-py3.7.egg\ttide\t_tide.py:345: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef = np.linalg.lstsq(lhs, rhs)[0].T
C:\Users\sergi\Anaconda3\lib\site-packages\numpy\lib\type_check.py:161: FutureWarning: `real` is deprecated and will be removed in a future version. To eliminate this warning for a Series `ser`, use `np.real(ser.to_numpy())` or `ser.to_numpy().real`.
  return val.real
C:\Users\sergi\Anaconda3\lib\site-packages\numpy\lib\type_check.py:204: FutureWarning: `imag` is deprecated and will be removed in a future version. To eliminate this warning for a Series `ser`, use `np.imag(ser.to_numpy())` or `ser.to_numpy().imag`.
  return val.imag


-----------------------------------
nobs = 1036988 
ngood = 1036987 
record length (days) = 720.13
rayleigh criterion = 1.0

Phases at central time
x0= 2.31  xtrend= 0
var(data)= 0.14    var(prediction)= 0.02    var(residual)= 0.13
var(prediction)/var(data) (%) = 13.5

        tidal amplitude and phase with 95 % CI estimates
 tide      freq        amp      amp_err   pha      pha_err    snr
* SA    0.0001141     0.0463     0.019    252.60     22.87       6.2
* SSA   0.0002282     0.0490     0.016     10.72     26.00       9.7
  MSM   0.0013098     0.0020     0.013     72.27    242.08     0.025
  MM    0.0015122     0.0091     0.015    320.36    130.43      0.38
  MSF   0.0028219     0.0019     0.012    151.32    258.94     0.025
  MF    0.0030501     0.0030     0.013    129.34    216.26     0.051
  ALP1  0.0343966     0.0007     0.016    102.01    265.45    0.0017
  2Q1   0.0357064     0.0012     0.016    339.14    220.77    0.0055
  SIG1  0.0359087     0.0020     0.014     34.07    204

C:\Users\sergi\Anaconda3\lib\site-packages\ttide-0.3_exp-py3.7.egg\ttide\t_utils.py:447: FutureWarning: `real` is deprecated and will be removed in a future version. To eliminate this warning for a Series `ser`, use `np.real(ser.to_numpy())` or `ser.to_numpy().real`.
  x = np.var(out['xingd'].real, ddof=1)
C:\Users\sergi\Anaconda3\lib\site-packages\ttide-0.3_exp-py3.7.egg\ttide\t_utils.py:449: FutureWarning: `real` is deprecated and will be removed in a future version. To eliminate this warning for a Series `ser`, use `np.real(ser.to_numpy())` or `ser.to_numpy().real`.
  xr = np.var(out['xresgd'].real, ddof=1)


In [33]:
out.keys()

dict_keys(['nobs', 'ngood', 'dt', 'xin', 'xout', 'xres', 'xingd', 'xoutgd', 'xresgd', 'isComplex', 'ray', 'nodcor', 'z0', 'dz0', 'fu', 'nameu', 'tidecon', 'snr', 'synth', 'lat', 'ltype', 'stime'])

In [107]:
out.get("stime")

In [43]:
out.get("dt")

0.016

In [41]:
out.t_predic()

TypeError: t_predic() missing 1 required positional argument: 'time'